<a href="https://colab.research.google.com/github/Strojove-uceni/23206-final-sign-language-recognition-in-video/blob/main/NN_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import math
import sys
import cv2
import io

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, LSTM, TimeDistributed
from sklearn.model_selection import train_test_split

from google.colab import drive

In [ ]:
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Colab Notebooks/SU2/data"
path_model = "/content/drive/MyDrive/Colab Notebooks/SU2/models"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# !git clone https://github.com/Strojove-uceni/23206-final-sign-language-recognition-in-video.git
!git clone https://github.com/jiri99/SU_sign-language.git

Cloning into 'SU_sign-language'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 14 (delta 5), reused 8 (delta 2), pack-reused 0
Receiving objects: 100% (14/14), 6.44 KiB | 6.44 MiB/s, done.
Resolving deltas: 100% (5/5), done.


In [ ]:
repo_path = "/content/23206-final-sign-language-recognition-in-video/"
sys.path.append(os.path.abspath(repo_path))

In [ ]:
df_train = pd.read_csv(path + "/train_mod.csv", sep=",")
df_train.head()

,Unnamed: 0,path,participant_id,sequence_id,sign,length_frames
0,0,train_landmark_files/26734/1000035562.parquet,26734,1000035562,blow,70.0
1,1,train_landmark_files/28656/1000106739.parquet,28656,1000106739,wait,34.0
2,2,train_landmark_files/16069/100015657.parquet,16069,100015657,cloud,426.0
3,3,train_landmark_files/25571/1000210073.parquet,25571,1000210073,bird,37.0
4,4,train_landmark_files/62590/1000240708.parquet,62590,1000240708,owie,55.0


In [ ]:
frame_height = 70
frame_width = 70
channels = 1
# classes_num = len(list(data_load.data.keys()))
classes_num = 10
sequence_length = 420

# CNN model for spatial feature extraction
cnn = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(frame_height, frame_width, channels)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten()
])

# Final model
model = Sequential([
    TimeDistributed(cnn, input_shape=(sequence_length, frame_height, frame_width, channels)),
    LSTM(64, return_sequences=False),
    Dense(64, activation='relu'),
    Dense(classes_num, activation='softmax')  # Assuming 10 classes for the classification
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.save(path_model + '/test_model')
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_2 (TimeDi  (None, 420, 6272)         92672     
 stributed)                                                      
                                                                 
 lstm_2 (LSTM)               (None, 64)                1622272   
                                                                 
 dense_4 (Dense)             (None, 64)                4160      
                                                                 
 dense_5 (Dense)             (None, 10)                650       
                                                                 
Total params: 1719754 (6.56 MB)
Trainable params: 1719754 (6.56 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
path_tensor = path + "/tensors"

labels = []
classes = {}
for idx, class_name in enumerate(sorted(os.listdir(path_tensor))):
    class_dir = os.path.join(path_tensor, class_name)
    if os.path.isdir(class_dir):
        classes[class_name] = {}
        classes[class_name]["path"] = class_dir
        classes[class_name]["id"] = idx
        labels.append(idx)

In [ ]:
classes

{'airplane': {'path': '/content/drive/MyDrive/Colab Notebooks/SU2/data/tensors/airplane',
  'id': 0},
 'apple': {'path': '/content/drive/MyDrive/Colab Notebooks/SU2/data/tensors/apple',
  'id': 1},
 'bad': {'path': '/content/drive/MyDrive/Colab Notebooks/SU2/data/tensors/bad',
  'id': 2},
 'brother': {'path': '/content/drive/MyDrive/Colab Notebooks/SU2/data/tensors/brother',
  'id': 3},
 'have': {'path': '/content/drive/MyDrive/Colab Notebooks/SU2/data/tensors/have',
  'id': 4},
 'man': {'path': '/content/drive/MyDrive/Colab Notebooks/SU2/data/tensors/man',
  'id': 5},
 'sad': {'path': '/content/drive/MyDrive/Colab Notebooks/SU2/data/tensors/sad',
  'id': 6},
 'wait': {'path': '/content/drive/MyDrive/Colab Notebooks/SU2/data/tensors/wait',
  'id': 7},
 'water': {'path': '/content/drive/MyDrive/Colab Notebooks/SU2/data/tensors/water',
  'id': 8},
 'zipper': {'path': '/content/drive/MyDrive/Colab Notebooks/SU2/data/tensors/zipper',
  'id': 9}}

In [ ]:
one_hot_encoded_train = pd.get_dummies(labels)
Y_onehot = tf.convert_to_tensor(one_hot_encoded_train, dtype=tf.float32)

for class_name in list(classes.keys()):
  classes[class_name]["Y"] = Y_onehot[classes[class_name]["id"]]

In [ ]:
classes["bad"]["Y"]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)>

In [ ]:
def run_learning(class_dict, model_path):
  stacked_data = []
  labels = []
  for file in os.listdir(class_dict["path"]):
    if file.endswith('.npy'):
      sample = np.load(os.path.join(class_dir, file))
      sample = np.expand_dims(sample.astype(np.float32), axis=3)
      stacked_data.append(sample)
      labels.append(class_dict["Y"])

  X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(stacked_data, labels, test_size=0.2, random_state=42)
  tensor_data = np.stack(X_train_raw, axis=0)
  X_train = tensor_data.reshape((len(X_train_raw), 420, 70, 70, 1))
  tensor_data = np.stack(X_test_raw, axis=0)
  X_test = tensor_data.reshape((len(X_test_raw), 420, 70, 70, 1))

  Y_train = np.stack(y_train_raw, axis=0)
  Y_test = np.stack(y_test_raw, axis=0)

  model = tf.keras.models.load_model(model_path)
  model.fit(X_train, Y_train, epochs=10, batch_size=1)
  model.save(model_path)

  tf.keras.backend.clear_session()

  return X_test, Y_test

KeyboardInterrupt: ignored

In [ ]:
X_test = []
Y_test = []

for class_name in list(classes.keys()):
  X_test_class, Y_test_class = run_learning(classes[class_name])
  X_test.append(X_test_class)
  Y_test.append(Y_test_class)

In [ ]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, Y_test, batch_size=1)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")

In [ ]:
# X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(stacked_data, labels, test_size=0.2, random_state=42)

ValueError: ignored

In [ ]:
# tensor_data = np.stack(X_train_raw, axis=0)
# X_train = tensor_data.reshape((len(X_train_raw), 420, 70, 70, 1))
# tensor_data = np.stack(X_test_raw, axis=0)
# X_test = tensor_data.reshape((len(X_test_raw), 420, 70, 70, 1))

NameError: ignored

In [ ]:
# # Y_train = np.stack(y_train_raw, axis=0)
# one_hot_encoded_train = pd.get_dummies(y_train_raw)
# Y_train = tf.convert_to_tensor(one_hot_encoded_train, dtype=tf.float32)
# one_hot_encoded_test = pd.get_dummies(y_test_raw)
# Y_test = tf.convert_to_tensor(one_hot_encoded_test, dtype=tf.float32)
# # Y_test = np.stack(y_test_raw, axis=0)

In [ ]:
# X_train.shape

In [ ]:
# Y_train.shape

In [ ]:
# stacked_data = []
# stacked_sign = []

# for key_sign in list(data_load.data.keys()):
#   for key_participiant in list(data_load.data[key_sign].keys()):
#     stacked_sign.append(key_sign)
#     stacked_data.append(data_load.data[key_sign][key_participiant].tensor)

# tensor_data = np.stack(stacked_data, axis=0)
# X_train = tensor_data.reshape((len(stacked_sign), 420, 70, 70, 1))
# stack_sign = np.stack(stacked_sign, axis=0)
# one_hot_encoded = pd.get_dummies(stack_sign)
# Y_train = tf.convert_to_tensor(one_hot_encoded, dtype=tf.float32)

In [ ]:
# Now you can fit this model on your preprocessed data.
# X_train would be your video data with shape (num_samples, sequence_length, frame_height, frame_width, channels)
# Y_train would be your labels
# model.fit(X_train, Y_train, epochs=10, batch_size=1)